# Chatbot use case : Flower commerce

Let's try to apply the knowledge we have so far on prompt engineering and Langchain to create a prototype of a chatbot. We will use a use case of flower commerce where customers can order flowers. This will include ability to extract intention and information from the prompt to be used in SQL query.

**NOTE**: This notebook provides examples on handling information extraction from prompts and data handling in database, which are presented in a way to make education easier, but **may not represent best practices nor production grade code**

## 1. Setup

In [ ]:
!pip install --upgrade -q pip
!pip install --upgrade -q langchain
!pip install -q transformers
!pip install -q faiss-gpu
!pip install -q bs4
!pip install -q sentence-transformers

In [ ]:
import regex
import json
import time
import sagemaker, boto3
import re
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

# secure the Sagemaker session, role, region, and other details to work with Sagemaker APIs
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sm_client = boto3.client("sagemaker", aws_region)
sess = sagemaker.Session()
model_version = "*"

Define method to invoke the SageMaker endpoint who hosts the LLM

In [ ]:
# functions that work with parsing JSON messages going to and from our Sagemaker Endpoint 
# (i.e. Deployed Sagemaker Jumpstart LLM) 

# invoke the Sagemaker Endpoint with the user query
def query_endpoint_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

# parse the Sagemaker Endpoint response to the user query
def parse_response_model(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    return [gen["generated_text"] for gen in model_predictions]

Configure the SageMaker Real-time Endpoint to use

In [ ]:
_MODEL_CONFIG_ = {
     "falcon-7b" : {
        "aws_region": "us-west-2",
        "endpoint_name": "jumpstart-dft-hf-llm-falcon-7b-instruct-bf16",
        "parse_function": parse_response_model
    },
}

Configure LangChain integration with SageMaker Endpoint

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

parameters ={
        "max_new_tokens": 100,
        "num_return_sequences": 1,
        "top_k": 1,
        "do_sample": False,
        "return_full_text": False,
        "temperature": 0.1,
        "stop": ["\n\n"]
}

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]


content_handler = ContentHandler()

# Passing the SagemakerEndPoint to LangChain so that it knows where to send all the inference requests
sm_llm_falcon_instruct = SagemakerEndpoint(
    endpoint_name=_MODEL_CONFIG_["falcon-7b"]["endpoint_name"],
    region_name=_MODEL_CONFIG_["falcon-7b"]["aws_region"],
    model_kwargs=parameters,
    content_handler=content_handler,
)

## 2. Configure database

Sometimes, after we capture the intention of our customer in our chatbot, we may need a connection to a database to serve the query. In this lab, we will build a simple shop assistant prototype which can connect to database to serve the intention captured by the LLM from the customer's dialog.

Now, let's configure the database. Let's pretend we are a flower shop which sells only 3 types of flowers for now: French Rose, Sunflower, and Dahlia. The database uses a local SQLite. It has 'flowers' and 'orders' table. We also define the method to create order.

In [ ]:
import sqlite3
from sqlite3 import Error

class DB():
    def __init__(self, file_name):
        self.file_name = file_name
        self.conn = None

    def create_connection(self):
        try:
            self.conn = sqlite3.connect(self.file_name)
        except Error as e:
            print(e)

        return self.conn
            
    def create_table(self, create_table_sql):
        if self.conn == None:
            self.conn = self.create_connection()
        try:
            c = self.conn.cursor()
            c.execute(create_table_sql)
        except Error as e:
            print(e)
        self.close_conn()

    def create_flowers_table(self):
        sql_create_flowers_table = """ CREATE TABLE IF NOT EXISTS flowers (
                                        id integer PRIMARY KEY AUTOINCREMENT,
                                        name text NOT NULL,
                                        unit_price text,
                                        stocks integer CHECK(stocks >= 0)
                                    ); """
        self.create_table(sql_create_flowers_table)

    def create_orders_table(self):
        sql_create_orders_table = """ CREATE TABLE IF NOT EXISTS orders (
                                        id integer PRIMARY KEY AUTOINCREMENT,
                                        flower_id integer NOT NULL,
                                        units integer,
                                        total_price real,
                                        time text NOT NULL,
                                        poem text,
                                        FOREIGN KEY (flower_id) REFERENCES flowers (id)
                                    ); """
        self.create_table(sql_create_orders_table)

    def insert_flowers(self):
        if self.conn == None:
            self.conn = self.create_connection()
            
        sql = ''' INSERT INTO flowers(name,unit_price,stocks)
                  VALUES ('French Rose',5.4,2), ('Sunflower',7.2,3), ('Dahlia',3.0,30), ('Lily',10.5,13)'''
        cur = self.conn.cursor()
        cur.execute(sql)
        self.conn.commit()
        last_row_id = cur.lastrowid
        self.close_conn()
        return last_row_id

    def create_order(self, flower: str, quantity: int) -> str:
        if self.conn == None:
            self.conn = self.create_connection()

        # Check first if the flower exists and whether there is stock
        sql = f"SELECT * FROM flowers WHERE name LIKE '{flower}';"
        print(f"""
            Running this SQL statement first:
            {sql}
        """)

        cur = self.conn.cursor()
        cur.execute(sql)
        row = cur.fetchone()
        print(f"""
            Results:
            id, name, unit_price, stocks
            {row}
        """)

        if row is None:
            return f"I am sorry that we do not sell {flower}. Please order Dahlia, Lily, French Rose, or Sunflower."
        else:
            flower_name = row[1]
            if (row[3]  - quantity) < 0:
                return f"I am sorry, there is no stock available for {quantity} of {flower_name}"
            else:
                total_price = float(quantity) * float(row[2])
                cur = self.conn.cursor()

                sql1 = f"""INSERT INTO orders(flower_id,units,total_price,time)
                          VALUES({row[0]},{quantity},{total_price},{int(time.time())});"""
                sql2 = f"""UPDATE flowers SET stocks = stocks - {quantity} WHERE id = {row[0]};"""
                print(f"""
                    Then running these SQL statements in a transaction:
                    {sql1}
                    {sql2}
                """)

                cur.execute(sql1)
                cur.execute(sql2)

                try:
                    self.conn.commit()
                    return f"The shop has {quantity} {flower_name} in stock. The order for {quantity} {flower_name} has been completed with order_flower tool"
                except Error as e:
                    print(e)
                    return "failed"
        self.close_conn()

    def view_orders(self):
        sql = f"SELECT * FROM orders;"
        print(f"Running this SQL statement: {sql}")
        self.conn = self.create_connection()
        cur = self.conn.cursor()
        rows = cur.execute(sql)
        print(f"Results:")
        print("id, flower_id, quantity, total_price, time, poem")
        for row in rows:
            print(row)
        self.close_conn()
    
    def add_poem(self, order_id, poem):
        if self.conn == None:
            self.conn = self.create_connection(self.file_name)

        # Check first if the flower exists and whether there is stock
        sql = f"UPDATE orders SET poem='{poem}' WHERE id={order_id};"
        cur = self.conn.cursor()
        cur.execute(sql)
        self.conn.commit()
        self.close_conn()
        return f"The poem is created and stored."

    def close_conn(self):
        self.conn.close()
        self.conn = None
            
db = DB("flowers.db")
db.create_flowers_table()
db.create_orders_table()
db.insert_flowers()
time.sleep(15)

## 3. Prompt engineering (few shots) for extracting both intention and order information

Let's first validate that the database does not have any order yet

Note: If you see any orders already, just delete the **flowers.db** file in this same directory and run the notebook again from top.

In [ ]:
db.view_orders()

Use few-shots technique to teach the LLM to extract the flower name and quantity requested from the conversation

We want the LLM to take the prompt and give us the extracted flower name and quantity being ordered in the form of (after post-processing):

```
{"intention": "order_flower", "flower": "Jasmine", "quantity": "1"}
```

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

prefix = """You are an intelligent flower shop assistant. Customer can order flowers. Extract the information on the flower name and quantity being ordered. 
Look at the examples below."""

examples = [
    {
        "history":"",
        "customer": "I buy 2 French roses. Please send tomorrow.",
        "extracted_information":  "{{\"intention\": \"order_flower\", \"flower\":\"French Rose\", \"quantity\":\"2\"}}"
    },
    {
        "history": "Customer Example: Let me order sunflowers." + \
                   "Shop AI Assistant: Sure. How many Sunflowers do you want to order?",
        "customer": "Four please.",
        "extracted_information":  "{{\"intention\": \"order_flower\", \"flower\":\"Sunflower\", \"quantity\":\"4\"}}"
    },
]

template = """{history}
Customer Example: {customer}
Extracted Information: {extracted_information}
"""
example_prompt = PromptTemplate(input_variables=["history","customer","extracted_information"], template=template)

suffix = """Given the examples above, extract the information from the customer's request / question below.

{history}
Customer: {input}
Extracted Information:"""

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    prefix=prefix,
    suffix=suffix, 
    input_variables=["history", "input"]
)

Let's define the method to ask the chatbot

In [ ]:
def ask_chatbot(history, input):
    entry = prompt.format(history=history,input=input)
    # Print the prompt
    print(entry)
    return sm_llm_falcon_instruct(entry)

Now let's ask the chatbot, which will call the Falcon-7b-instruct model in SageMaker

In [ ]:
history = ""
input = "Hello, can I get 2 Dahlia please?"

response = ask_chatbot(history, input)
response

Let's try one where the information is spread in the history

In [ ]:
history = """
Customer: Please send me Dahlia flowers tomorrow.
Shop AI Assistant: Sure. How many?"""

input = "Two please."

response = ask_chatbot(history, input)
response

Now, let's extract the information from the JSON into a Python's dictionary so that information can be passed on to the SQL query more easily.

In [ ]:
def extract_json_from_answer(answer):
    answer = answer.replace("\n","\\n")
    pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
    task_json = json.loads(pattern.search(answer).group(0))
    return task_json
task = extract_json_from_answer(response)
task

## 4. Make database change based on extracted information

Now that we know what to be ordered, let's make the SQL query to simulate a real purchase

In [ ]:
db.create_order(task['flower'], int(task['quantity']))

In [ ]:
db.view_orders()

## 5. Integrating with LangChain's tools and agent

So far we have been using prompts only. Now we want to use LangChain's agent to try automating the intent and information extraction and driving the completion until we have an answer for the customer.

### Defining agent's prompt

Let's modify the previous few-shots prompt template to adopt the ReAct framework for the agent.

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

SYSTEM_MESSAGE = """You are an intelligent flower shop assistant who assists Customer to order flower or do other things.
Flower order must have both flower name and flower quantity. If one or more information is missing, ask the customer back.
When there is no stock, do not continue the order. When we do not sell the flower, do not continue the order."""

few_shot_template_prefix = """{system_message}

You have access to these tools:
{tools_information}

Use the following format:

Customer: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Shop AI Assistant: the final answer to the original input question

See the examples below:"""

few_shots = [
    {
        "history":"",
        "customer": "I buy 2 French roses. Please send tomorrow.",
        "agent_scratchpad":  "Thought: The Customer wants to order flower[French Rose] with quantity[2]. All information is complete. I can proceed with creating order with order_flower tool.\n" + \
                             "Action: order_flower\n" + \
                             "Action Inputs: French Rose, 2\n" + \
                             "Observation: The shop has 2 French Rose in stock. The order for 2 French Rose has been completed with order_flower tool.\n" + \
                             "Thought: Now I know how to respond to the Customer\n" +\
                             "Shop AI Assistant: Your order for 2 French Rose has been successfully completed. Thank you for shopping with us.\n"
    },
    {
        "history": "Customer: Let me order sunflowers.\n" + \
                   "Shop AI Assistant: Sure. How many Sunflowers do you want to order?",
        "customer": "Four please.",
        "agent_scratchpad":  "Thought: The Customer wants to order flower[Sunflower] with quantity[4]. All information is complete. I can proceed with creating order with order_flower tool.\n" + \
                             "Action: order_flower\n" + \
                             "Action Inputs: Sunflower, 4\n" + \
                             "Observation: The shop has 4 Sunflower in stock. The order for 4 Sunflower has been completed with order_flower tool.\n" + \
                             "Thought: Now I know how to respond to the Customer\n" +\
                             "Shop AI Assistant: Your order for 4 Sunflower has been successfully completed. Have a nice day!\n"
    },
    {
        "history": "",
        "customer": "Can I get 10 jasmines?",
        "agent_scratchpad":  "Thought: The Customer wants to order flower[Jasmine] with quantity[10]. All information is complete. I can proceed with creating order with order_flower tool.\n" + \
                             "Action: order_flower\n" + \
                             "Action Inputs: Jasmine, 10\n" + \
                             "Observation: I am sorry, there is no stock available for 10 of Jasmine.\n" + \
                             "Thought: Now I know how to respond to the customer\n" +\
                             "Shop AI Assistant: I am sorry, we do not have enough stock for 10 of Jasmine flowers.\n"
    }
]

each_shot_template = """{history}
Customer: {customer}
{agent_scratchpad}
"""

each_shot_prompt_template = PromptTemplate(input_variables=["history","customer","agent_scratchpad"], template=each_shot_template)

few_shot_template_suffix = """Given the examples above, assist the Customer based on the recent conversation below.

{history}
Customer: {input}
{agent_scratchpad}"""

agent_template = FewShotPromptTemplate(
    examples=few_shots, 
    example_prompt=each_shot_prompt_template, 
    prefix=few_shot_template_prefix,
    suffix=few_shot_template_suffix, 
    input_variables=["history","agent_scratchpad","input","tools_information", "tool_names", "system_message"]
)

Redefine a simple method to invoke the LLM using the LangChain prompt

In [ ]:
tools_information = """
order_flower: Useful for completing the flowers order.
"""
tool_names = "order_flower"
STOP_TOKENS = ["\nObservation:"]
sm_llm_falcon_instruct.model_kwargs["stop"] = STOP_TOKENS
def ask_chatbot(history, agent_scratchpad, input):
    entry = agent_template.format(history=history,agent_scratchpad=agent_scratchpad,input=input, tools_information=tools_information, tool_names=tool_names, system_message=SYSTEM_MESSAGE)
    # Print the prompt
    print(entry)
    return sm_llm_falcon_instruct(entry)

Test how the LLM now behaves with the new prompt template. First we test with a scenario where customer's information is complete and order is to be made using order_flower tool.

In [ ]:
history = ""
agent_scratchpad =  ""
input = "Can I get 2 Dahlia flowers please?"

response = ask_chatbot(history, agent_scratchpad, input)
response

Now assume the agent has managed to call the order_flower tool successfully and the ordered is inserted into the database. It should now update the customer.

### Creating tools

Let's now create the LangChain tools for fulfilling the intentions

In [ ]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

def order_flower(string):
    """"useful for when you need to make an order to buy lily flowers. 
    The input to this tool should be a comma separated list string and number of length two, representing the flower and quantity you want to buy. 
    For example, `Lily, 3` would be the input if you wanted to make an order for Lily flowers of quantity 3."""
    flower, quantity = string.split(",")
    return db.create_order(flower.strip(), int(quantity.strip()))

tools = [
    Tool(
        name="order_flower",
        func=order_flower,
        description="Useful for completing the flowers order"
    )
]

### Creating Langchain Agent

In [ ]:
from typing import Union
from langchain.agents import AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        print(f'llm_output={llm_output}')
            
        if "Shop AI Assistant" in llm_output:
            try: 
                lines = llm_output.split("\n")
                for line in lines:
                    if "Shop AI Assistant" in line:
                        llm_output = line
                        break
            except:
                pass
            return AgentFinish(
                return_values={"output": llm_output.split("Shop AI Assistant:")[-1].strip()},
                log=llm_output,
            )
        
        if "Action Inputs" in llm_output:
            regex = r"Action\s*\d*\s*:(.*?)[\n]Action\s*\d*\s*Inputs\s*\d*\s*:[\s]*(.*)"
            match = re.search(regex, llm_output, re.DOTALL)
            if not match:
                raise ValueError(f"Could not parse LLM output: `{llm_output}`")

            action = match.group(1).strip()
            action_input = match.group(2).strip()
            return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

        raise ValueError(f"Could not parse LLM output: `{llm_output}`")

Next, we will also define the prompt template to handle the "intermediate_steps" variable defined in the agent.

In [ ]:
from langchain.prompts import StringPromptTemplate

class CustomPromptTemplate(StringPromptTemplate):
    template: FewShotPromptTemplate
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        print(f"intermediate_steps:{intermediate_steps}")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            #thoughts += f"\nObservation: {observation}\nThought: "
            thoughts += f"\nObservation: {observation}"
        
        kwargs["agent_scratchpad"] = thoughts
        
        kwargs["tools_information"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        kwargs["system_message"] = SYSTEM_MESSAGE

        return self.template.format(**kwargs)
    
agent_prompt = CustomPromptTemplate(
    template=agent_template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

Next we define the chain and the agent. We use memory to allow the prompt to extract information from the chat history.

In [ ]:
from langchain import LLMChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory


llm_chain = LLMChain(llm=sm_llm_falcon_instruct, prompt=agent_prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=CustomOutputParser(),
    stop=STOP_TOKENS, 
    allowed_tools=[tool.name for tool in tools]
)


memory = ConversationBufferMemory(
    memory_key="history",
    human_prefix="Customer",
    ai_prefix="Shop AI Assistant"
)

agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)
agent_chain.agent.llm_chain.verbose=True


Let's try the agent. With the agent, now the call to the LLM, the information extraction, and the fulfillment of the intent (e.g. inserting data to DB) can be done in one step from the user perspective.

In [ ]:
agent_chain.run(input="Can I get 11 pieces of lily for next week?")

Let's verify by looking at the 'orders' table

In [ ]:
db.view_orders()

Now let's try to order again when it is already out of stock

In [ ]:
agent_chain.run(input="Can I get 11 lily for next week?")

Let's now challenge it with ability to collect more information when not present.

In [ ]:
agent_chain.run(input="Let me order blue orchids")

Let's try a more advanced scenario where the flower name and quantity are not in 1 message. Rather they are spread in the conversation.

## 5. Add 'create poem' intent by leveraging LLM's generative capability

As the title implies, let's add the third intent to create poem. This also demonstrates the LLM's creative generative capability

First, we add a new example in the prompt's few shots, to reflect the intention to create poem and add it to a flower order.

In [ ]:
few_shots.append({
    "history":"",
    "customer": "Please add a poem of how much I love my mother in her mother's day. My order_id is 1.",
    "agent_scratchpad":  "Thought: The Customer requested a poem for order_id[1] and provided the topic. I can continue with create_poem tool.\n" + \
                         "Action: create_poem\n" + \
                         "Action Inputs: 1, how much I love my mother in her mother's day.\n" + \
                         "Observation: The poem is created and stored.\n" + \
                         "Thought: Now I know how to respond to the Customer\n" +\
                         "Shop AI Assistant: Your poem has been added to the order. Check your order to see the poem.\n"
})

Then we define the query to the database, the function to fulfil the intent, and reinitialize the agent chain.

In [ ]:
def create_poem(string):
    poem_args = string.split(",")
    order_id = poem_args[0]
    topic = ",".join(poem_args[1:])
    prompt = f"""
    Write a poem about how much I love my mother in her mother's day.   
    Poem: Upon your gentle care, I softly lay\\nUpon your heart, I would peacefully starate\\nTo think of what you've done, I'd count each day\\nIn gratefulness, I'd thank you, my mother, for being there.
    
    Write a poem about {topic}
    Poem: 
    """
    initial_args = sm_llm_falcon_instruct.model_kwargs.copy()
    sm_llm_falcon_instruct.model_kwargs["temperature"] = 0.9
    sm_llm_falcon_instruct.model_kwargs["max_new_token"] = 50
    sm_llm_falcon_instruct.model_kwargs["stop"] = ["\n\n\n"]
    
    poem = sm_llm_falcon_instruct(prompt)
    
    sm_llm_falcon_instruct.model_kwargs = initial_args.copy()
    
    poem = poem.replace("'","''")
    print(f"""Poem generated:
        {poem}
    """)
    print("Adding poem to database")
    return db.add_poem(order_id, poem)

tools.append(Tool(
    name="create_poem",
    func=create_poem,
    description="Useful for creating poem"
))

agent_template = FewShotPromptTemplate(
    examples=few_shots, 
    example_prompt=each_shot_prompt_template, 
    prefix=few_shot_template_prefix,
    suffix=few_shot_template_suffix, 
    input_variables=["history","agent_scratchpad","input","tools_information", "tool_names", "system_message"]
)

agent_prompt = CustomPromptTemplate(
    template=agent_template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=sm_llm_falcon_instruct, prompt=agent_prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=CustomOutputParser(),
    stop=STOP_TOKENS, 
    allowed_tools=[tool.name for tool in tools]
)


memory = ConversationBufferMemory(
    memory_key="history",
    human_prefix="Customer",
    ai_prefix="Shop AI Assistant"
)

agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)
agent_chain.agent.llm_chain.verbose=True

Let's test it out.

In [ ]:
agent_chain.run(input="Please add a poem of how I am thankful to my teacher, Rendy. My order_id is 1.")

Of course we will define the prompt template and the few-shots example.

Now let's ask the LLM again to create the poem then store it in the database

Let's check the orders table to see if the poem is already added.

In [ ]:
db.view_orders()

## 6. Add RAG

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

# Define the txt2embedding model
sm_llm_embeddings = HuggingFaceEmbeddings()

# Load documents
loader = TextLoader(file_path="./Flower_Shop_FAQs.txt")
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Embedd your texts
docsearch = FAISS.from_documents(texts, sm_llm_embeddings)

In [ ]:
few_shots.append({
    "history":"",
    "customer": "Do you do international delivery?",
    "agent_scratchpad":  "Thought: The Customer asked a question about shop FAQs. I can continue with shop_faqs tool.\n" + \
                         "Action: shop_faqs\n" + \
                         "Action Inputs: Do you do international delivery?\n" + \
                         "Observation: No, only local delivery is supported\n" + \
                         "Thought: Now I know how to respond to the Customer\n" +\
                         "Shop AI Assistant: I am sorry, we only do local delivery for now."
})

In [ ]:
def ask_faqs(question):
    docs = docsearch.similarity_search(question, k=1)
    docs_string = "\n\n".join(list(map(lambda x: x.page_content, docs)))
    print("\nDocuments found:")
    print(docs_string)
    prompt = f"""
    ===Context===
    {docs_string}
    ===End of Context===
    Given the context above, answer the question below. ONLY use information from the context. Do not make up any new information.
    Question: {question}
    Answer:"""
    print("\nRAG prompt:")
    print(prompt)
    initial_args = sm_llm_falcon_instruct.model_kwargs.copy()
    sm_llm_falcon_instruct.model_kwargs["max_new_token"] = 50
    sm_llm_falcon_instruct.model_kwargs["stop"] = ["\n"]

    answer = sm_llm_falcon_instruct(prompt)
    
    sm_llm_falcon_instruct.model_kwargs = initial_args.copy()
    
    return answer

tools.append(Tool(
    name="shop_faqs",
    func=ask_faqs,
    description="Useful for answering questions about this purchase methods, return policy, delivery, and related matters."
))

agent_template = FewShotPromptTemplate(
    examples=few_shots, 
    example_prompt=each_shot_prompt_template, 
    prefix=few_shot_template_prefix,
    suffix=few_shot_template_suffix, 
    input_variables=["history","agent_scratchpad","input","tools_information", "tool_names", "system_message"]
)

agent_prompt = CustomPromptTemplate(
    template=agent_template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=sm_llm_falcon_instruct, prompt=agent_prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=CustomOutputParser(),
    stop=STOP_TOKENS, 
    allowed_tools=[tool.name for tool in tools]
)


memory = ConversationBufferMemory(
    memory_key="history",
    human_prefix="Customer",
    ai_prefix="Shop AI Assistant"
)

agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)
agent_chain.agent.llm_chain.verbose=True

In [ ]:
agent_chain.run(input="Do you provide international delivery?")

## 7. UX

Let's have a simple UX

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

from io import StringIO
import sys
import textwrap

def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"Shop AI Assistant:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="Customer:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Now feel free to play around with the chat input. Remember, it uses memory, so it may load your earlier chat as well.

You will notice that sometimes the result is not what we expected. We can probably improve this by adding more few-shots, or fine-tuning the model, or just better prompt. Feel free to explore!

In [ ]:
chat = ChatUX(agent_chain)
chat.start_chat()

## 8. Clean up

Please refer to the **Lab 1 - Cleanup section of the workshop's instruction**. Especially, you need to delete the SageMaker endpoint you spun up for hosting the Falcon-7b-instruct model we used in the workshop so far